In [1]:
import itertools

from collections import deque
from dataclasses import dataclass

In [2]:
spells = {
    "magic_missile": {
        "mana_cost": 53,
        "damage": 4,
    },
    "drain": {
        "mana_cost": 73,
        "damage": 2,
        "heals": 2
    },
    "shield": {
        "mana_cost": 113,
        "armor": 7,
        "lasts": 6
    },
    "poison": {
        "mana_cost": 173,
        "damage": 3,
        "lasts": 6
    },
    "recharge": {
        "mana_cost": 229,
        "mana_gain": 101,
        "lasts": 5
    },
}

In [3]:
day22_inputs = "inputs/day22.txt"
with open(day22_inputs, 'r') as file:
    day22_data = file.readlines()
day22_data_split = [el.strip().split(": ") for el in day22_data]
day22_data_flat = list(itertools.chain.from_iterable(day22_data_split))

boss = {day22_data_flat[i]: int(day22_data_flat[i + 1]) for i in range(0, len(day22_data_flat), 2)}

print(boss)

{'Hit Points': 55, 'Damage': 8}


In [4]:
# constructing a breadth first search
@dataclass
class GameState:
    player_hp: int
    player_mana: int
    player_armor: int
    active_effects: dict
    boss_hp: int
    mana_spent: int
    turn: str

    # turn into a hash value, and use a frozenset to be immutable
    def __hash__(self):
        return hash((
            self.player_hp,
            self.player_mana,
            self.player_armor,
            frozenset(self.active_effects.items()),
            self.boss_hp,
            self.turn
        ))

    # check if two game states are equal 
    def __eq__(self, other):
        if not isinstance(other, GameState):
            return False
        return (
            self.player_hp == other.player_hp and
            self.player_mana == other.player_mana and
            self.player_armor == other.player_armor and
            self.active_effects == other.active_effects and
            self.boss_hp == other.boss_hp and
            self.turn == other.turn
        )


In [5]:
def apply_effects(state):
    curr_player_hp = state.player_hp
    curr_player_mana = state.player_mana
    curr_player_armor = 0
    new_effects = {}
    curr_boss_hp = state.boss_hp

    for effect, timer in state.active_effects.items():
        if effect == "recharge":
            curr_player_mana += spells[effect]["mana_gain"]
        if effect == "poison":
            curr_boss_hp -= spells[effect]["damage"]
        if effect == "shield":
            curr_player_armor = spells[effect]["armor"]
        
        if timer > 1:
            new_effects[effect] = timer - 1

    return GameState(
        player_hp=curr_player_hp,
        player_mana=curr_player_mana,
        player_armor=curr_player_armor,
        active_effects=new_effects,
        boss_hp=curr_boss_hp,
        mana_spent=state.mana_spent,
        turn=state.turn
    )

In [18]:
def apply_player_spell(state, spell_name, spell_data):
    # copy current state
    curr_player_hp = state.player_hp
    curr_player_mana = state.player_mana
    curr_player_armor = state.player_armor
    new_effects = state.active_effects.copy()
    curr_boss_hp = state.boss_hp
    curr_mana_spent = state.mana_spent

    # deduct mana cost, and track spent mana
    curr_player_mana -= spell_data["mana_cost"]
    curr_mana_spent += spell_data["mana_cost"]

    # apply spell
    if "lasts" in spell_data:  # effect spell
        new_effects[spell_name] = spell_data["lasts"]
        if spell_name == "shield":
            curr_player_armor = spell_data["armor"]
    else:  # instant effect spell
        curr_boss_hp -= spell_data.get("damage", 0)
        if spell_name == "drain":
            curr_player_hp += spell_data.get("heals", 0)

    return GameState(
        player_hp=curr_player_hp,
        player_mana=curr_player_mana,
        player_armor=curr_player_armor,
        active_effects=new_effects,
        boss_hp=curr_boss_hp,
        mana_spent=curr_mana_spent,
        turn="boss" # next turn is boss
    )

def apply_boss_attack(state, boss):
    # copy current state
    curr_player_hp = state.player_hp
    curr_player_mana = state.player_mana
    curr_player_armor = state.player_armor
    new_effects = state.active_effects.copy()
    curr_boss_hp = state.boss_hp
    curr_mana_spent = state.mana_spent

    boss_damage = max(1, boss["Damage"] - curr_player_armor)
    curr_player_hp -= boss_damage

    next_turn = "player" # next turn is player

    return GameState(
        player_hp=curr_player_hp,
        player_mana=curr_player_mana,
        player_armor=curr_player_armor,
        active_effects=new_effects,
        boss_hp=curr_boss_hp,
        turn=next_turn,
        mana_spent=curr_mana_spent
    )


In [19]:
def generate_next_states(state):
    next_states = []
    
    if state.turn == "player":
        for spell_name, spell_data in spells.items():
            # only cast next spell if enough mana and not existing effect spell
            if (spell_data["mana_cost"] <= state.player_mana) and (spell_name not in state.active_effects):
                new_state = apply_player_spell(state, spell_name, spell_data)
                next_states.append(new_state)
    elif state.turn == "boss":
        new_state = apply_boss_attack(state, boss)
        next_states.append(new_state)
        
    return next_states

In [20]:
def is_goal_state(state):
    # success if boss dies and player is still alive
    return state.boss_hp <= 0 and state.player_hp > 0

def is_dead_state(state):
    # failure if player has died
    return state.player_hp <= 0

# part1

In [21]:
def bfs_min_mana(initial_state):
    queue = deque([initial_state])
    
    # what spell combos have been attempted
    visited = {}
    visited[initial_state] = 0
    
    # track the mana 
    min_mana_to_win = float('inf')
    
    while queue:
        state = queue.popleft()

        if state.mana_spent >= min_mana_to_win:
            continue

        state = apply_effects(state)

        if is_goal_state(state):
            min_mana_to_win = min(min_mana_to_win, state.mana_spent)
            continue
        if is_dead_state(state):
            continue

        for next_state in generate_next_states(state):
            key = next_state
            if key in visited and visited[key] <= next_state.mana_spent:
                continue
            visited[key] = next_state.mana_spent
            queue.append(next_state)

    return min_mana_to_win if min_mana_to_win != float('inf') else -1

In [22]:
initial_state = GameState(
    player_hp=50,
    player_mana=500,
    boss_hp=boss["Hit Points"],
    player_armor=0,
    active_effects={},
    mana_spent=0,
    turn="player" # player starts
)

result = bfs_min_mana(initial_state)
print(f"minimum mana to win is {result}")

minimum mana to win is 953


# part2

In [35]:
def player_lose_health(state):
    curr_player_hp = state.player_hp
    curr_player_mana = state.player_mana
    curr_player_armor = state.player_armor
    curr_boss_hp = state.boss_hp
    new_effects = state.active_effects.copy()
    curr_mana_spent = state.mana_spent

    curr_player_hp -= 1
    
    return GameState(
        player_hp=curr_player_hp,
        player_mana=curr_player_mana,
        player_armor=curr_player_armor,
        boss_hp=curr_boss_hp,
        active_effects=new_effects,
        turn=state.turn,
        mana_spent=curr_mana_spent
    )


In [38]:
def bfs_min_mana_part2(initial_state):
    queue = deque([initial_state])
    
    # what spell combos have been attempted
    visited = {}
    visited[initial_state] = 0
    
    # track the mana 
    min_mana_to_win = float('inf')
    
    while queue:
        state = queue.popleft()
        
        if state.turn == "player":
            state = player_lose_health(state)
            # check if player has died
            if is_dead_state(state):
                continue
        
        state = apply_effects(state)

        if is_goal_state(state):
            min_mana_to_win = min(min_mana_to_win, state.mana_spent)
            continue
        if is_dead_state(state):
            continue

        for next_state in generate_next_states(state):
            key = next_state
            if key in visited and visited[key] <= next_state.mana_spent:
                continue
            visited[key] = next_state.mana_spent
            queue.append(next_state)

    return min_mana_to_win if min_mana_to_win != float('inf') else -1

In [39]:
initial_state = GameState(
    player_hp=50,
    player_mana=500,
    boss_hp=boss["Hit Points"],
    player_armor=0,
    active_effects={},
    mana_spent=0,
    turn="player" # player starts
)

result = bfs_min_mana_part2(initial_state)
print(f"minimum mana to win is {result}")

minimum mana to win is 1289
